In [1]:
import os
from glob import glob
import math

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import skimage.exposure
from scipy import ndimage
from scipy import signal

import cv2 as cv
from PIL import Image, ImageOps

### Create X and y

In [122]:
data_type = 'Data218' 
base_dir = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence'
save = True

data_dir = os.path.join(os.path.join(base_dir, 'data'), 'Edge')
fig_dir = os.path.join(os.path.join(base_dir, 'figures'), 'Edge')
if save:
  crop_fig_dir = os.path.join(os.path.join(base_dir, 'figures'), 'Crop')
os.makedirs(os.path.join(fig_dir, data_type), exist_ok=True)
os.makedirs(os.path.join(crop_fig_dir, data_type), exist_ok=True)

PATH = os.path.join(data_dir, data_type)
os.chdir(PATH)

for (i, filename) in enumerate(sorted(glob('*.tif'))):
  image = Image.open(os.path.join(PATH, filename)).convert("L")

  width, height = image.size
  left = 5
  top = 30
  right = width - left
  bottom = (right - left) + top
  image = image.crop((left, top, right, bottom))
  image = np.array(image)
  image = np.expand_dims(image, axis=0)

  if save:
    colored_image = Image.open(os.path.join(PATH, filename)).convert('RGBA')
    colored_image = colored_image.crop((left, top, right, bottom))

    datas = colored_image.getdata()
    newData = []
 
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((255, 255, 255, 0))
        else:
            newData.append((item[0], item[1], item[2], 130))
 
    colored_image.putdata(newData)
    frame_num = filename.split('.')[0]
    colored_image.save(os.path.join(os.path.join(crop_fig_dir, data_type), f'{frame_num}.png'))


  if i == 0:
    X = image
    h,  w = X.shape[1], X.shape[2]
  else:
    image = image[:, :h, :w]
    X = np.concatenate([X, image], axis=0)

y = np.array([0] * X.shape[0])
print(X.shape, y.shape)
print(np.unique(y))
print(X.min(), X.max(), y.min(), y.max())

(8, 322, 322) (8,)
[0]
0 116 0 0


### Detect edges

In [60]:
color_type = 'rainbow'

if color_type == 'greens':
    color_dict = {0: (204, 224, 148),
                  1: (159, 196, 51),
                  2: (117, 170, 1),
                  3: (72, 142, 1),
                  4: (27, 114, 1),
                  5: (1, 88, 1),
                  6: (1, 60, 1), 
                  7: (1, 32, 1)}
elif color_type == 'rainbow':
    color_dict = {0: (140, 1, 1),
                  1: (255, 14, 1),
                  2: (254, 157, 1),
                  3: (229, 198, 1),
                  4: (1, 229, 1),
                  5: (1, 171, 255),
                  6: (1, 27, 255), 
                  7: (110, 1, 199)}

def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb
hex_color_dict = {i: rgb_to_hex(rgb) for (i, rgb) in color_dict.items()}

In [61]:
for i in range(X.shape[0]):
  img = X[i, :, :]

  # zero-pad to detect borderline edges later on
  padded = np.pad(img, ((3, 3), (3, 3)), 'constant', constant_values=0)

  # get the outer borderline only
  binary = np.where(padded > 0, 255, padded) # the most outer boarder

  # edge detect with Canny operator
  binary = np.uint8(binary)
  canny = cv.Canny(binary, 100, 200)

  # make it into binary image
  binary2 = np.where(canny < 128, 0, canny)
  binary2 = np.where(canny >= 128, 255, canny)

  # smooth the edge
  blur = cv.GaussianBlur(binary2, ksize=(0, 0), sigmaX=1, sigmaY=1, borderType = cv.BORDER_DEFAULT) 

  # make it into binary image
  binary3 = np.where(blur > 0, 255, blur)

  # to make colorful contour lines
  colored = np.expand_dims(binary3, axis=-1)
  colored = np.concatenate([colored, colored, colored], axis=-1)
  
  colored = np.where(colored==(255, 255, 255), color_dict[i], colored)
  colored = colored / 255. # change it to float

  alphas = np.ones((colored.shape[0], colored.shape[1]))
  alphas = np.where(colored[:, :, 0] == 0, 0, alphas)
  alphas = np.expand_dims(alphas, axis=-1)

  colored = np.concatenate([colored, alphas], axis=-1)

  # thin out the edges
  kernel = np.ones((3, 3), np.uint8)
  eroded = cv.erode(colored, kernel, iterations=1)

  # visualize 
  plt.imshow(eroded, cmap='gray') 
  plt.axis('off') 
  plt.xticks([])
  plt.yticks([])
  # plt.show()

  # save the image
  fname = os.path.join(os.path.join(os.path.join(fig_dir, f'{data_type}'), f'{i+10}54.png'))
  plt.savefig(fname, transparent=True)
  plt.clf()

<Figure size 640x480 with 0 Axes>